In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content


In [ ]:
import pandas as pd
import os
images_list=os.listdir('/content/drive/MyDrive/685/emogen/test/images')
mapping_csv = pd.read_csv('/content/drive/MyDrive/685/emogen/test/caption/test_caption.csv')
#mapping_csv.set_index('Sno',inplace=True)
count=0
rem_count=0
for i,row in mapping_csv.iterrows():
  count+=1
  if str(row['image_hash']+'.jpg') not in images_list:
    #print(row)
    print(i)
    mapping_csv.drop(mapping_csv.index[mapping_csv['image_hash'] == row['image_hash']],inplace=True)
    
    rem_count+=1

print(mapping_csv)
print(count,rem_count)
mapping_csv.reset_index(drop=True)
mapping_csv.to_csv('/content/drive/MyDrive/685/Project/root/ParlAI/emogen/test/caption/test.csv')

     Unnamed: 0  ...                        image_hash
0             0  ...  be877e8b958869c6751578f137139d7c
1             1  ...  43998d5eee707c76ae879c3bf733764b
2             2  ...  4b4d5bce7238f027615e497fd86ca5b3
3             3  ...   ebd4282f96cb5dc31342826f292e0da
4             4  ...  ce17d5e8b351b8e54b3927ccb351f2a2
..          ...  ...                               ...
471         471  ...    187dbeb0d95c31cc7e0b2c11556888
472         472  ...  637a5ecb1a8ad062e5bd9366b8d7fbde
473         473  ...   b9eeda9ec527bf93a1a38b632fc9f97
474         474  ...   c5249fbf2dc5ce47d3b2224f8c8855b
475         475  ...  fc826b6646344d16cfc9fc33b5dc824b

[476 rows x 7 columns]
476 0


In [ ]:
%pip install transformers
import os
import torch
import random
import warnings
import numpy as np
import transformers
import pandas as pd 
from PIL import Image
import torch.nn as nn
warnings.filterwarnings("ignore")
import torchvision.transforms as T
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from typing import Callable, Optional


## For Reproducibility
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

## Tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)

## Device Configuration 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     |████████████████████████████████| 3.1 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 47.0 MB/s 
     |████████████████████████████████| 61 kB 508 kB/s 
     |████████████████████████████████| 3.3 MB 34.1 MB/s 
     |████████████████████████████████| 596 kB 32.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def load_data(data_path,data_type):
  csv_str =str("caption/"+data_type+"_"+"caption.csv")
  img_path = os.path.join(data_path,data_type,'images/')
  mapping_csv = pd.read_csv(os.path.join(data_path,data_type,csv_str))
  print(len(mapping_csv.index))
  image_name = {
    'image_name':img_path+mapping_csv['image_hash']+'.jpg'
  }
  
  captions = {
      'caption':mapping_csv['comment']
  }
  image_name_df = pd.DataFrame.from_dict(image_name)
  captions_df = pd.DataFrame.from_dict(captions)
  if data_type!="train":
    candidates = {
        'candidates':mapping_csv['candidates']
    }
    candidates_df = pd.DataFrame.from_dict(candidates)
    df = pd.concat([image_name_df,captions_df,candidates_df], axis=1)
  else:
    df = pd.concat([image_name_df,captions_df], axis=1)
  #Image.open(df['image_name'][0]) To debug if Image path is correct
  return df

In [ ]:
data_dir = '/content/drive/MyDrive/685/emogen'
train=load_data(data_dir,"train")
val=load_data(data_dir,"val")
test = load_data(data_dir,"test")
print(train.shape)
print(val.shape)
print(test.shape)

/content


FileNotFoundError: ignored

In [ ]:
class DatasetPrep(Dataset):
    
    def __init__(self, df, 
                 transforms: Optional[Callable] = None,name=None) -> None:
        self.df = df
        self.name=name
        self.transforms = T.Compose([
            T.ToTensor(),
            T.Normalize(mean = [0.5], std = [0.5]),
            T.Resize((256,256)),
        ])
        
    def __len__(self) -> int:
        return len(self.df)
    
    def __getitem__(self, idx: int):
        
        image_id = self.df.image_name.values[idx]
        image = Image.open(image_id).convert('RGB')
            
        if self.transforms is not None:
            image = self.transforms(image)
            
        comments = self.df[self.df.image_name == image_id].values.tolist()[0][1:][0]
        encoded_inputs = tokenizer(comments,
                                   return_token_type_ids = False, 
                                   return_attention_mask = False, 
                                   max_length = 100, 
                                   padding = "max_length",
                                  return_tensors = "pt")
        
        sample = {"image":image.to(device),"captions": encoded_inputs["input_ids"].flatten().to(device)}
        if self.name!="train":
          candidates=self.df[self.df.image_name == image_id].values.tolist()[0][1:]#[1:]
          print(candidates)
          encoded_cand = tokenizer(candidates,
                                   return_token_type_ids = False, 
                                   return_attention_mask = False, 
                                   max_length = 100, 
                                   padding = "max_length")
          sample["candidates"] = encoded_cand["input_ids"]
          #print(sample["candidates"])
          
        

        return sample

In [ ]:
batch_size = 4

train_dataset = DatasetPrep(train, transforms = True,name="train")

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, drop_last=True)

val_dataset = DatasetPrep(val, transforms = True,name="val")
print(val_dataset.__getitem__(2))
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size,drop_last=True)

test_dataset = DatasetPrep(test, transforms = True,name="test")
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size,drop_last=True)

NameError: ignored

In [ ]:
class CNN(nn.Module):
    
    def __init__(self, embed_size):
        super(CNN, self).__init__()
        model = models.resnet50(pretrained=True)
        for param in model.parameters():
            param.requires_grad_(False)
        
        modules = list(model.children())[:-1]
        self.model = nn.Sequential(*modules)
        self.embed = nn.Linear(model.fc.in_features, embed_size)
        
    def forward(self, image):
        features = self.model(image)
        features = features.view(features.size(0), -1)
        features = self.embed(features)
                
        return features

In [ ]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_dim,vocab_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        
        self.embedding = nn.Embedding(num_embeddings = vocab_size,embedding_dim = embedding_dim)
        
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            batch_first=True)
        
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    def init_hidden(self, features):
        
        return (torch.autograd.Variable(torch.zeros(1,4,512).to(device)), 
                torch.autograd.Variable(features.unsqueeze(0)).to(device))
        
    def forward(self, features, captions):
        
        state = self.init_hidden(features)
        
        embed = self.embedding(captions)
                    
        lstm_out, state = self.lstm(embed, state)
                        
        outputs = self.fc(lstm_out)
        outputs = outputs.view(-1, self.vocab_size)
        
        return outputs

In [ ]:
example_batch = next(iter(train_dataloader))

image, captions = example_batch["image"], example_batch["captions"]

encoder = CNN(embed_size = 512).to(device)
decoder = RNN(input_size = 512, hidden_size = 512, embedding_dim=512, vocab_size = 28881).to(device)

features = encoder(image)
embed = decoder(features, captions)

print("Image Transformation: ", image.shape, " --> ", features.shape)
print("Captions Transformation: ", captions.shape, " --> ", embed.shape)

Image Transformation:  torch.Size([4, 3, 256, 256])  -->  torch.Size([4, 512])
Captions Transformation:  torch.Size([4, 100])  -->  torch.Size([400, 28881])


In [ ]:
vocab_size = 90000
steps_per_epoch = 19069 // 4

encoder = CNN(embed_size = 512).to(device)
decoder = RNN(input_size = 512, hidden_size = 512, embedding_dim=512, vocab_size = vocab_size).to(device)

criterion = nn.CrossEntropyLoss().to(device)
params = list(decoder.parameters()) + list(encoder.embed.parameters())

optimizer = torch.optim.Adam(params, lr=0.001)

In [ ]:
def accuracy(scores, targets, k):
    """
    Computes top-k accuracy, from predicted and true labels.
    :param scores: scores from the model
    :param targets: true labels
    :param k: k in top-k accuracy
    :return: top-k accuracy
    """

    batch_size = targets.size(0)
    _, ind = scores.topk(k, 1, True, True)
    correct = ind.eq(targets.view(-1, 1).expand_as(ind))
    correct_total = correct.view(-1).float().sum()  # 0D tensor
    return correct_total.item() * (100.0 / batch_size)

In [ ]:
def validate(val_loader,encoder,decoder,criterion):
  encoder.eval()
  decoder.eval()
  for idx, sample in enumerate(val_dataloader):
    print(sample.keys())
    image, captions = torch.tensor(sample['image']).to(device), torch.tensor(sample['captions']).to(device)#,torch.tensor(sample['candidates']).to(device)
    features = encoder(image)
    outputs = decoder(features, captions)
    loss = criterion(scores, targets)
    print(len(candidates))
    _, preds = torch.max(scores_copy, dim=2)
    preds = preds.tolist()
    hypotheses.extend(preds)

    assert len(references) == len(hypotheses)

    # Calculate BLEU-4 scores
    bleu4 = corpus_bleu(references, hypotheses)

    return bleu4


In [ ]:
for epoch in range(10):
    print("Epoch ",epoch)

    for idx, sample in enumerate(train_dataloader):
        
        if idx > steps_per_epoch:
            break
        
        image, captions = torch.tensor(sample['image']).to(device), torch.tensor(sample['captions']).to(device)
        
        # zero the parameter gradients
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Forward pass
        features = encoder(image)
        outputs = decoder(features, captions)

        
        # Compute the Loss
        loss = criterion(outputs.view(-1, vocab_size), 
                         captions.view(-1))
        #acc = accuracy(outputs,captions.view(-1),5)
        bleu = validate(val_dataloader,encoder,decoder,criterion)
        
        # Backward pass.
        loss.backward()

        
        # Update the parameters in the optimizer.
        optimizer.step()
        state = {'epoch': epoch,
             'encoder': encoder,
             'decoder': decoder,
             'optimizer': optimizer}
        filename = 'checkpoint_' + '.pth.tar'
    print(str("loss"+str(loss)))
    torch.save(state, filename)


Epoch  0
['[\'Such an exquisite sunrise \', "I don\'t know what is going on here but I think this guy might be on TV.", \'What I would give to be in her position!\', \'This man is intent on this wall.\', \'The colors are just so beautiful, and peaceful.\', "Are those rocks about to fall or is it just me. I don\'t know how I feel about this picture/ ", \'Music will fix it all Bro! \', \'The haze on this picture ruins the colors\', \'I would hate to have to walk this street - it looks long.\', \'This sign is like, totally ridiculous.\', \'I wish I lived in a place with so much greenery\', \'Machines are taking over the world.\', \'This is such a lovely color.\', \'The dinner was very lovely and casual.\', \'Creative, uncontrollable, colorful\', \'We can see there are several varieties of stones and fish\', \'My country, right or wrong, it is still my country.\', \'What a lovely plant.\', \'your hair is so fluffy\', \'thats a nice photo\', \'I would get naked and run out on to that field.

RuntimeError: ignored